In [2]:
from dota.parse import get_response
from dota.persistence import load, save
from dota.data import json_to_dataframe, default_name

import bs4
import numpy as np
import pandas as pd
import time

from sklearn.metrics import accuracy_score

In [3]:
with open('data/heroes.txt', 'r') as f:
    all_heroes = f.read().split("\n")
    all_heroes.remove('')

In [ ]:
hero_rates = load('data/hero_diffs.pkl')

In [7]:
data = load('data/18.08-3.09_lower.pkl')
df = json_to_dataframe(data)
df = df[['picks_0', 'picks_1', 'winner']]

In [19]:
X = []
for i, row in df.iterrows():
    winner = row['winner']
    vector = []
    for pick in row['picks_0']:
        pick = default_name(pick)
        for pick_2 in row['picks_1']:
            pick_2 = default_name(pick_2)
            vector.append(hero_rates[pick][pick_2])
    X.append(vector)

In [29]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

cross_val_score(estimator=xgb.XGBClassifier(), X=pd.DataFrame(X), y=y,
                cv=5, scoring='accuracy')

/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: Use

[01:30:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:30:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:30:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:30:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

array([0.48913043, 0.55434783, 0.42391304, 0.40217391, 0.56043956])

#### А если просуммировать, и просто глянуть, насколько велика разница? 

In [24]:
X = np.array(X)

In [21]:
y = df['winner']

In [163]:
indexes_we_sure = [i for i, x in enumerate(abs(np.array(X).mean(axis=1)) > 0.24) if x]

In [164]:
len(indexes_we_sure)

115

In [165]:
accuracy_score(y[indexes_we_sure], (X[indexes_we_sure].sum(axis=1) > 0).astype(int))

0.6086956521739131